In [34]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import pandas as pd
import numpy as np
import json
import os
import sklearn
import sklearn.neighbors
from pathlib import Path
from pandas.api.types import CategoricalDtype
from operator import itemgetter

import warnings
warnings.filterwarnings('ignore')

# constants
RAW_DATA_PATH = Path('raw_data/')
DATAFRAME_PATH = Path('dataframes/')
MODEL_PATH = Path('model/state_dict_model.pt')
TOTAL_TRACKS = 50
NUM_WITHHELD = 25
SELECTED_TRACK_FEATURES = ['danceability', 'energy', 'key', 'loudness',
                     'speechiness', 'acousticness', 'instrumentalness',
                     'liveness', 'valence', 'tempo']

In [2]:
# read tracks.h5 which contains tracks with features like danceability, loudness ,... etc
# set type of track_uri to category, to convert string to an int unique id
# rename track_uri to tid and sort values based on tid
tracks_features_df = pd.read_hdf(DATAFRAME_PATH / 'tracks.h5', 'tracks')
cat_type = CategoricalDtype(categories=tracks_features_df.track_uri, ordered=True)
tracks_features_df.track_uri = tracks_features_df.track_uri.astype(cat_type)
tracks_features_df = tracks_features_df.rename(columns={'track_uri':'tid'})
tracks_features_df = tracks_features_df[['tid',*SELECTED_TRACK_FEATURES]].sort_values('tid').reset_index(drop=True)
tracks_features_df.head(5)

,tid,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,spotify:track:3uvsVUrAaGQJCTEUR1S3Sx,0.523,0.422,3,-10.385,0.0270,0.78300,0.005140,0.0898,0.145,129.876
1,spotify:track:0heE5tAAaDQmnGhVDImPl2,0.493,0.594,8,-4.404,0.0378,0.25600,0.000000,0.0759,0.304,123.751
2,spotify:track:3omXshBamrREltcf24gYDC,0.468,0.692,2,-4.015,0.0295,0.02020,0.000002,0.5230,0.561,78.009
3,spotify:track:6TYWE19e35N7Bn5heHwyY6,0.553,0.564,9,-7.072,0.0418,0.03900,0.000000,0.3180,0.310,149.953
4,spotify:track:1xznGGDReH1oQq0xzbwXa3,0.791,0.619,1,-5.886,0.0532,0.00784,0.004230,0.3510,0.371,103.989


In [3]:
# read 10000 playlists from the official spoitfy 1M playlists
# for each playlist, get the tracks and their positions
def make_playlist_dfs(path):
    playlists = []
    playlist_tracks = []
    for file in tqdm(os.listdir(path)):
        if not file.endswith(".json"):
            continue
        with open(path/file) as f:
            js_slice = json.load(f)
            for playlist in js_slice['playlists']:
                if playlist['num_tracks'] > TOTAL_TRACKS:
                    sorted_tracks = sorted(playlist['tracks'], key=itemgetter('pos')) 
                    for track in sorted_tracks[:TOTAL_TRACKS]:
                        yield track['track_uri'], playlist['pid'], track['pos']

playlist_tracks_df = pd.DataFrame(make_playlist_dfs(RAW_DATA_PATH),columns=['tid','pid','pos'])
playlist_tracks_df.tid = playlist_tracks_df.tid.astype(cat_type)
playlist_tracks_df.head(5)

100%|██████████| 12/12 [00:02<00:00,  5.03it/s]


,tid,pid,pos
0,spotify:track:3uvsVUrAaGQJCTEUR1S3Sx,7000,0
1,spotify:track:0heE5tAAaDQmnGhVDImPl2,7000,1
2,spotify:track:3omXshBamrREltcf24gYDC,7000,2
3,spotify:track:6TYWE19e35N7Bn5heHwyY6,7000,3
4,spotify:track:1xznGGDReH1oQq0xzbwXa3,7000,4


In [4]:
all_playlist_ids = playlist_tracks_df.pid.unique()
train_pids, test_pids = train_test_split(all_playlist_ids,random_state=0, test_size=0.20)
print(f'total playlists: {len(all_playlist_ids)}')
print(f'train playlists: {len(train_pids)}')
print(f'test playlists: {len(test_pids)}')
# 1. Get tracks that are only from the training playlists
# 2. Get the track features(danceability,loudness) for each of these training tracks
# 3. Make a test set that only includes tracks from the training set
train_playlist_tracks_df = playlist_tracks_df.query('pid in @train_pids')

unique_train_tracks = train_playlist_tracks_df['tid'].unique()
train_cat_type = CategoricalDtype(categories=unique_train_tracks, ordered=True)
train_playlist_tracks_df.tid = train_playlist_tracks_df.tid.astype(train_cat_type)

train_tracks_features_df = tracks_features_df.query('tid in @train_playlist_tracks_df.tid')
test_playlist_tracks_df = playlist_tracks_df.query('pid in @test_pids and tid in @train_playlist_tracks_df.tid')
print()
print(f'total tracks in train playlists: {len(train_playlist_tracks_df)}')
print(f'unique tracks in train playlists: {len(train_tracks_features_df)}')


# 1. Get the first NUM_WITHHELD tracks for each playlist in test
# 2. Get the track features for these with held tracks
# 3. Compute mean features by grouping the tracks from incomplete playlists
test_playlist_tracks_incomplete_df = test_playlist_tracks_df.groupby('pid').head(NUM_WITHHELD)
test_tracks_incomplete_features_df = test_playlist_tracks_incomplete_df.merge(tracks_features_df,how='inner',on='tid')
test_playlist_incomplete_features = test_tracks_incomplete_features_df[['pid',*SELECTED_TRACK_FEATURES]].groupby('pid',as_index=False).mean()
print()
print(f'total tracks in test playlists: {len(test_playlist_tracks_df)}')
print(f'total tracks in incomplete test playlists: {len(test_playlist_tracks_incomplete_df)}')
print(f'total incomplete test playlists: {len(test_playlist_incomplete_features)}')

total playlists: 4907
train playlists: 3925
test playlists: 982

total tracks in train playlists: 196250
unique tracks in train playlists: 71782

total tracks in test playlists: 36534
total tracks in incomplete test playlists: 22919
total incomplete test playlists: 975


In [5]:
# KNN model that will find 25 nearest neighbors to the current playlist
knn_clf = sklearn.neighbors.NearestNeighbors(n_neighbors=NUM_WITHHELD)
knn_clf.fit(train_tracks_features_df[SELECTED_TRACK_FEATURES])
distances, indices = knn_clf.kneighbors(test_playlist_incomplete_features[SELECTED_TRACK_FEATURES])

In [6]:
# for each test playlist, get the 25 next nearest predicted tracks and add them to a table for evaluation
def get_predicted_playlist_tracks():
    for index, row in test_playlist_incomplete_features.iterrows():
            predicted_tracks = train_tracks_features_df['tid'].iloc[indices[index]]
            for pos, predicted_track in enumerate(predicted_tracks):
                yield predicted_track, int(row['pid']),pos

test_predicted_playlist_tracks_df = pd.DataFrame(get_predicted_playlist_tracks(), columns =['tid', 'pid', 'pos'])
test_predicted_playlist_tracks_df

,tid,pid,pos
0,spotify:track:0lCG1ndJE7vsNrpWgTv17L,0,0
1,spotify:track:1U37ItHkYDpKMVvoIPcxGc,0,1
2,spotify:track:3LSN1ZAbeXmYWyq4O1jRGM,0,2
3,spotify:track:03HpzzH36FRs8zYONAwFJb,0,3
4,spotify:track:6QBkPN3ARmAFVtVAk7oYN9,0,4
...,...,...,...
24370,spotify:track:6eHhubs2FeAVjWkSRRMLRS,9973,20
24371,spotify:track:6808fuj56HYDOND2CpyqsT,9973,21
24372,spotify:track:1K0VI2vWPR85jvGeeWPIjD,9973,22
24373,spotify:track:6khitcV9375KZ5hZRrSiL0,9973,23


## Neural Network (NeuMF)

In [7]:
from scipy.sparse import dok_matrix
import torch

In [8]:
train_playlist_tracks_df['cat_codes'] = train_playlist_tracks_df['tid'].cat.codes # Add a new column for integral values
unique_train_tracks = train_playlist_tracks_df['cat_codes']

# Make a dictionary key sparse matrix
dok_mat = dok_matrix((train_playlist_tracks_df.shape[0], len(unique_train_tracks)))

# TODO vectorize later if possible
for (pid, cat_code) in tqdm(zip(train_playlist_tracks_df['pid'], train_playlist_tracks_df['cat_codes'])):
    dok_mat[pid, cat_code] = 1.0

196250it [00:02, 76179.60it/s]


In [29]:
# Hyperparameters
embedding_dim = 8
num_epochs = 2
learning_rate = 0.001
batch_size = 256
rand_negative_fill_in = 4
layer_sizes = [64, 32, 16, 8]

In [30]:
class NeuralMF(torch.nn.Module):
    def __init__(self, num_pl, num_tr, dim):
        super(NeuralMF, self).__init__()
        num_of_layers = len(layer_sizes)
            
        self.pl_embedding = torch.nn.Embedding(num_pl, dim, sparse=True)
        self.tr_embedding = torch.nn.Embedding(num_tr, dim, sparse=True)
        
        self.pl_mlp_embedding = torch.nn.Embedding(num_pl, layer_sizes[0] // 2)
        self.tr_mlp_embedding = torch.nn.Embedding(num_tr, layer_sizes[0] // 2)
        
        def lecunn_uniform(layer):
            fan_in, fan_out = layer.in_features, layer.out_features
            limit = np.sqrt(3. / fan_in)
            layer.weight.data.uniform_(-limit, limit)
        def glorot_uniform(layer):
            fan_in, fan_out = layer.in_features, layer.out_features
            limit = np.sqrt(6. / (fan_in + fan_out))
            layer.weight.data.uniform_(-limit, limit)
            
        self.mlp = torch.nn.ModuleList()
        for i in range(1, num_of_layers):
            self.mlp.extend([torch.nn.Linear(layer_sizes[i - 1], layer_sizes[i])])
        for layer in self.mlp:
            glorot_uniform(layer)

        self.final = torch.nn.Linear(dim + layer_sizes[-1], 1)
        lecunn_uniform(self.final)

    def forward(self, playlists, tracks):
        pl_vec = self.pl_embedding(playlists)
        tr_vec = self.tr_embedding(tracks)
        prod = pl_vec*tr_vec
        
        pl_mlp_vec = self.pl_mlp_embedding(playlists)
        tr_mlp_vec = self.tr_mlp_embedding(tracks)
        
        mlp_vec = torch.cat([pl_mlp_vec, tr_mlp_vec], dim=-1)
        
        for i, layer in enumerate(self.mlp):
            mlp_vec = layer(mlp_vec)
            mlp_vec = torch.nn.functional.relu(mlp_vec)
            
        result = self.final(torch.cat([prod,mlp_vec], dim=-1))
        activated_result = torch.flatten(torch.sigmoid(result))
        return activated_result

In [31]:
class SparseDenseOptimizer:
    def __init__(self, *op):
        self.optimizers = op

    def zero_grad(self):
        for op in self.optimizers:
            op.zero_grad()

    def step(self):
        for op in self.optimizers:
            op.step()

In [32]:
model = NeuralMF(dok_mat.shape[0], dok_mat.shape[1], embedding_dim)

dense_weights_to_optimize = [weight_tensor for (param_name, weight_tensor) in list(model.named_parameters()) if param_name not in ('pl_embedding.weight','tr_embedding.weight')]
sparse_weights_to_optimize = [model.pl_embedding.weight,model.tr_embedding.weight]
sparse_optim = torch.optim.SparseAdam(sparse_weights_to_optimize, lr=learning_rate)
dense_optim = torch.optim.Adam(dense_weights_to_optimize, lr=learning_rate)
optim = SparseDenseOptimizer(sparse_optim, dense_optim)

assert len(dense_weights_to_optimize + sparse_weights_to_optimize) == len(list(model.parameters()))

loss_fn = torch.nn.BCEWithLogitsLoss()

def make_train_data():
    pl_input, tr_input, recs = [], [], []
    for (pl, tr) in dok_mat.keys():
        yield pl, tr, 1.0
        for t in range(rand_negative_fill_in):
            rand_num = np.random.randint(dok_mat.shape[1])
            while(pl, rand_num) in dok_mat.keys():
                rand_num = np.random.randint(dok_mat.shape[1])
            yield pl, rand_num, 0.0

def train_loop(data_loader):
    for epoch in range(num_epochs):
        for pl, tr, recs in data_loader:
            optim.zero_grad()
            out = model(pl, tr)
            loss = loss_fn(out, recs)
            loss.backward()
            optim.step()
        print(f'training loss after epoch-{epoch+1} = {(loss.item()*100):.4f}%')

def run_training():
    data_loader = torch.utils.data.DataLoader(list(make_train_data()), batch_size=batch_size)
    train_loop(data_loader)

In [33]:
run_training()

training loss after epoch-1 = 69.3148%
training loss after epoch-2 = 69.3147%


NameError: name 'MODEL_PATH' is not defined

In [35]:
torch.save(model.state_dict(), MODEL_PATH)
model = NeuralMF(dok_mat.shape[0], dok_mat.shape[1], embedding_dim)
model.load_state_dict(torch.load(MODEL_PATH))

<All keys matched successfully>

In [ ]:
# code to make recommendations